# Magic Formula - Modelo de Investimento com Backtest

## Desafio:

- Testar se a regra de investimento da fórmula mágica de Joel Grenblatt funcionou no Brasil nos últimos anos.

## Passo a passo:

* Passo 1 - Importar as bibliotecas;
* Passo 2 - Baixar os dados disponibilizados;
* Passo 3 - Pegar as datas que iremos balancear a carteira;
* Passo 4 - Filtrar a liquidez;
* Passo 5 - Criar o ranking dos indicadores;
* Passo 6 - Criar as carteiras;
* Passo 7 - Calcular a rentabilidade por carteira;
* Passo 8 - Carcular a rentabilidade do modelo;
* Passo 9 - Calcular a rentabilidade do ibovespa no mesmo período;
* Passo 10 - Analisar os resultados.

## Passo 1: Importando os módulos necessários

In [1]:
import pandas as pd
import quantstats as qs

## Passo 2: Baixar os dados disponibilizados

In [2]:
# Dados dos ativos desde 2013

dados_empresas = pd.read_csv('dados_empresas.csv')

## Passo 3: Filtrar liquidez

In [3]:
# Filtrar empresas que negociam a partir de 1 milhão por dia

dados_empresas = dados_empresas[dados_empresas['volume_negociado'] > 1000000]
dados_empresas

,data,ticker,preco_fechamento_ajustado,volume_negociado,ebit_ev,roic
0,2016-12-31,AALR3,14.45,5375545.0,0.08785,0.05890
1,2017-01-31,AALR3,13.75,3116406.0,0.09044,0.05890
2,2017-02-28,AALR3,12.56,3077805.0,0.09519,0.05890
3,2017-03-31,AALR3,15.41,3141709.0,0.04781,0.05517
4,2017-04-30,AALR3,18.32,6444929.0,0.04118,0.05517
...,...,...,...,...,...,...
49268,2023-04-30,ZAMP3,4.20,4243398.0,0.05860,0.03526
49269,2023-05-31,ZAMP3,3.81,4414483.0,0.05600,0.03131
49270,2023-06-30,ZAMP3,4.55,17615157.0,0.04998,0.03131
49271,2023-07-31,ZAMP3,4.42,4634150.0,0.05094,0.03131


## Passo 4 - Calcular os retornos mensais das empresas

In [4]:
# calcular o retorno mensal de cada empresa
dados_empresas['retorno'] = dados_empresas.groupby('ticker')['preco_fechamento_ajustado'].pct_change()

# Alocando o resultado do retorno médio no início do período (mês anterior), em outras palavras,
# mês de criação da carteira.
# Exemplo: cria-se uma carteira em 31/12/2020, esta carteira é mensal, logo, o seu resultado será
# no dia 31/01/2021, ou seja, para sabermos o retorno da carteira de dezembro, pegaremos o cálculo
# do fim de janeiro e aplicá-lo em dezembro.

dados_empresas['retorno'] = dados_empresas['retorno'].shift(-1)
dados_empresas

,data,ticker,preco_fechamento_ajustado,volume_negociado,ebit_ev,roic,retorno
0,2016-12-31,AALR3,14.45,5375545.0,0.08785,0.05890,-0.048443
1,2017-01-31,AALR3,13.75,3116406.0,0.09044,0.05890,-0.086545
2,2017-02-28,AALR3,12.56,3077805.0,0.09519,0.05890,0.226911
3,2017-03-31,AALR3,15.41,3141709.0,0.04781,0.05517,0.188838
4,2017-04-30,AALR3,18.32,6444929.0,0.04118,0.05517,-0.045306
...,...,...,...,...,...,...,...
49268,2023-04-30,ZAMP3,4.20,4243398.0,0.05860,0.03526,-0.092857
49269,2023-05-31,ZAMP3,3.81,4414483.0,0.05600,0.03131,0.194226
49270,2023-06-30,ZAMP3,4.55,17615157.0,0.04998,0.03131,-0.028571
49271,2023-07-31,ZAMP3,4.42,4634150.0,0.05094,0.03131,0.255656


## Passo 5 - Criar o ranking dos indicadores

In [5]:
dados_empresas['ranking_ebit_ev'] = dados_empresas.groupby('data')['ebit_ev'].rank(ascending = False)
dados_empresas['ranking_roic'] = dados_empresas.groupby('data')['roic'].rank(ascending = False)

dados_empresas

,data,ticker,preco_fechamento_ajustado,volume_negociado,ebit_ev,roic,retorno,ranking_ebit_ev,ranking_roic
0,2016-12-31,AALR3,14.45,5375545.0,0.08785,0.05890,-0.048443,40.0,61.0
1,2017-01-31,AALR3,13.75,3116406.0,0.09044,0.05890,-0.086545,33.0,63.0
2,2017-02-28,AALR3,12.56,3077805.0,0.09519,0.05890,0.226911,32.0,67.0
3,2017-03-31,AALR3,15.41,3141709.0,0.04781,0.05517,0.188838,78.0,77.0
4,2017-04-30,AALR3,18.32,6444929.0,0.04118,0.05517,-0.045306,85.0,76.0
...,...,...,...,...,...,...,...,...,...
49268,2023-04-30,ZAMP3,4.20,4243398.0,0.05860,0.03526,-0.092857,161.0,167.0
49269,2023-05-31,ZAMP3,3.81,4414483.0,0.05600,0.03131,0.194226,155.0,170.0
49270,2023-06-30,ZAMP3,4.55,17615157.0,0.04998,0.03131,-0.028571,165.0,179.0
49271,2023-07-31,ZAMP3,4.42,4634150.0,0.05094,0.03131,0.255656,160.0,177.0


In [11]:
dados_empresas['ranking_final'] = dados_empresas['ranking_ebit_ev'] + dados_empresas['ranking_roic']
dados_empresas['ranking_final'] = dados_empresas.groupby('data')['ranking_final'].rank()

In [12]:
dados_empresas

,data,ticker,preco_fechamento_ajustado,volume_negociado,ebit_ev,roic,retorno,ranking_ebit_ev,ranking_roic,ranking_final
0,2016-12-31,AALR3,14.45,5375545.0,0.08785,0.05890,-0.048443,40.0,61.0,50.5
1,2017-01-31,AALR3,13.75,3116406.0,0.09044,0.05890,-0.086545,33.0,63.0,47.0
2,2017-02-28,AALR3,12.56,3077805.0,0.09519,0.05890,0.226911,32.0,67.0,51.0
3,2017-03-31,AALR3,15.41,3141709.0,0.04781,0.05517,0.188838,78.0,77.0,80.5
4,2017-04-30,AALR3,18.32,6444929.0,0.04118,0.05517,-0.045306,85.0,76.0,87.5
...,...,...,...,...,...,...,...,...,...,...
49268,2023-04-30,ZAMP3,4.20,4243398.0,0.05860,0.03526,-0.092857,161.0,167.0,175.0
49269,2023-05-31,ZAMP3,3.81,4414483.0,0.05600,0.03131,0.194226,155.0,170.0,171.0
49270,2023-06-30,ZAMP3,4.55,17615157.0,0.04998,0.03131,-0.028571,165.0,179.0,181.5
49271,2023-07-31,ZAMP3,4.42,4634150.0,0.05094,0.03131,0.255656,160.0,177.0,177.0
